In [1]:
import pandas as pd
import os
from glob import glob

In [2]:
#新增資料夾，放置天氣資料
path='./weather'
if not os.path.exists(path):
    os.mkdir(path)

In [3]:
#預先清洗天氣資料
count=[]
for i in range(1,13):
    count.append('%02d'%i)
def get_data(z='2020',x=count):
    for month in x:
        df=pd.read_csv(f'466920-{z}-{month}.csv')
        df=df.drop(['測站氣壓(hPa)','海平面氣壓(hPa)','測站最高氣壓(hPa)','測站最高氣壓時間(LST)','測站最低氣壓(hPa)',
            '測站最低氣壓時間(LST)','最高氣溫(℃)','最高氣溫時間(LST)','最低氣溫(℃)','最低氣溫時間(LST)',
            '最小相對溼度(%)','最小相對溼度時間(LST)','風向(360degree)','最大陣風(m/s)','最大陣風風向(360degree)'
            ,'最大陣風風速時間(LST)','降水時數(hour)','最大十分鐘降水量(mm)','最大十分鐘降水量起始時間(LST)',
            '最大六十分鐘降水量(mm)','最大六十分鐘降水量起始時間(LST)', 'A型蒸發量(mm)','日照時數(hour)',
            '總雲量(0~10)','日照率(%)','全天空日射量(MJ/㎡)','能見度(km)','日最高紫外線指數時間(LST)'],axis=1)
        df=df.drop(0,axis=0)
        g=df.loc[:,'降水量(mm)'].values=='T'
        for i,j in enumerate (g):
            if j == True:
                df.iloc[i,5]=0.1
        df.to_csv(f'{path}/{z}-{month}.csv',encoding='utf-8-sig',index=0)
    print('save!')

In [4]:
get_data()
get_data('2021')
get_data('2022',x=count[:5])

save!
save!
save!


In [5]:
files = glob('weather/202*.csv') #合併29筆天氣資料
df = pd.concat(
    (pd.read_csv(file, usecols=['觀測時間(day)','氣溫(℃)','露點溫度(℃)','相對溼度(%)','風速(m/s)','降水量(mm)','日最高紫外線指數'],) for file in files), ignore_index=True)
df

,觀測時間(day),氣溫(℃),露點溫度(℃),相對溼度(%),風速(m/s),降水量(mm),日最高紫外線指數
0,1,13.9,6.5,63,2.9,0.0,7
1,2,17.7,9.8,62,2.0,0.0,6
2,3,17.9,12.9,74,1.4,0.1,6
3,4,17.1,15.7,92,1.3,8.1,2
4,5,16.0,13.4,85,2.8,6.8,2
...,...,...,...,...,...,...,...
877,26,23.9,21.3,86,2.3,2.5,3
878,27,21.6,21.5,99,2.1,39.5,2
879,28,22.5,22.5,100,1.9,16.5,3
880,29,24.5,20.2,79,2.2,1.0,7


In [6]:
#合併天氣、假日、確診人數資料，填補缺值
df=df.drop('觀測時間(day)',axis=1)
df1=pd.read_csv('time.csv',encoding='utf-8-sig',index_col=0)
weather=pd.concat([df1,df],axis=1)
totalPassenger=pd.read_csv('totalPassenger123.csv')
totalPassenger=pd.DataFrame(totalPassenger['passengers'],columns=['passengers'])
w_t=pd.concat([weather,totalPassenger],ignore_index=True,axis=1)
workcale=pd.read_csv('109_111_workcale.csv')
ww=pd.concat([w_t,workcale['是否放假']],ignore_index=True,axis=1)
ww.set_index(0,inplace = True)
covid=pd.read_csv('109_111_covid.csv',index_col=0)
data=pd.concat([ww,covid], axis=1, sort=False)
data=data.drop(['區域'],axis=1)
final_data=data.fillna(0)
final_data.reset_index(inplace=True)
final_data.columns=['觀測時間(day)','氣溫(℃)','露點溫度(℃)','相對溼度(%)','風速(m/s)','降水量(mm)','日最高紫外線指數','日客流量','0:工作日 1:假日','新增確診人數']
# final_data

In [7]:
final_data.to_csv('final_data.csv',encoding='utf-8-sig') #輸出資料